## Syntax-dependent trace

Head attribution

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=3

!echo $CUDA_VISIBLE_DEVICES

env: CUDA_VISIBLE_DEVICES=3
3


In [2]:
## discovery cluster setup
# CACHE_DIR = "/scratch/lucchetti.f/models/"
# os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

import os
import sys
sys.path.append('..')

import random
import gc
import torch
import pandas as pd
from model_utils import *
import tqdm as notebook_tqdm
from trace_model import ModelLoader

torch.set_grad_enabled(False)
from tqdm import tqdm 
from model_utils import layername
import numpy as np
from extract_data import *

MODEL_NAME = "bigcode/starcoder"
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import re
check_devs()
# clear_devs()
torch.cuda.is_available()

/home/franlucc/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 84978434048 used for device 0, reserved 0


True

In [3]:
lm = ModelLoader(MODEL_NAME)
check_devs()

Loading checkpoint shards: 100%|██████████| 7/7 [00:30<00:00,  4.36s/it]


62136934400 / 84978434048 used for device 0, reserved 62138613760


## int var/val experiment

In [4]:
from datasets import *

# val_dataset = load_from_disk("../data/py_int_value_funcs.parquet")["python"]
var_dataset = load_from_disk("../data/py_int_var_funcs_large.parquet")["examples"]

print("\n".join(var_dataset[0].split("\n")))

def mover_alfil(tablero, x_inicial, y_inicial, x_final, y_final):
    """
    (list of list, int, int, int, int) -> list of list

    :param tablero: list of list que representa el tablero
    :param x_inicial: int que representa la posicion inicial en X
    :param y_inicial: int que representa la posicion inicial en Y
    :param x_final: int que representa la posicion final en X
    :param y_final: int que representa la posicion final en Y
    :return: list of list que representa un tablero final
    """
    tab = tablero.copy()
    if ((x_inicial - y_inicial == x_final - y_final) or (x_inicial + y_inicial == x_final + y_final)) and tab[x_inicial][y_final].lower() == 'a':
        if (x_inicial != x_final) and (y_inicial != y_final):
            for x in range(x_inicial +1, x_final):
                if tab[x][y_final] != ' ':
                    raise ValueError('El camino no es valido')
            for y in range(y_inicial +1, y_final):
                if tab[x_final][y] != ' ':
     

In [6]:
x = 2
print(var_dataset[x])
print("\n",crop_to_int_var(var_dataset[x], crop_before_int=True))

def memdiff_search(bytes1, bytes2):
    """
    Use binary searching to find the offset of the first difference
     between two strings.

    :param bytes1: The original sequence of bytes
    :param bytes2: A sequence of bytes to compare with bytes1
    :type bytes1: str
    :type bytes2: str
    :rtype: int offset of the first location a and b differ, None if strings match
    """

    # Prevent infinite recursion on inputs with length of one
    half = (len(bytes1) // 2) or 1

    # Compare first half of the string
    if bytes1[:half] != bytes2[:half]:

        # Have we found the first diff?
        if bytes1[0] != bytes2[0]:
            return 0

        return memdiff_search(bytes1[:half], bytes2[:half])

    # Compare second half of the string
    if bytes1[half:] != bytes2[half:]:
        return memdiff_search(bytes1[half:], bytes2[half:]) + half

 (None, 'bytes1, bytes2):\n    """\n    Use binary searching to find the offset of the first difference\n     between two strings.\

In [7]:
# prompts = [crop_to_int_var(var_dataset[0])]
# print(prompts[0])
# txt, ret_dict = lm.trace_generate(
#     prompts,
#     max_new_toks= 1,
#     pick_greedily= True,
#     request_activations= [], # lm.model.config.n_layer
#     request_logits= [layername(lm.model, 6)], #lm.model.config.n_layer
#     report_topk_logits = 10
# )

# txt, ret_dict["logits"][layername(lm.model, 6)]

## Experiment: 6th layer can predict int

Using python data

In [15]:
results = {}
for i,ex in tqdm(enumerate(var_dataset[:100])):
    # crop_to_int = "\n".join(ex.split("\n")[-3:])
    # crop_to_int = re.findall(r"([\w\W]*?)\d", crop_to_int)[0]
    # print(crop_to_int)
    prompt, var_name = crop_to_int_var(ex, crop_before_int=True)
    if prompt == None or var_name == None:
        continue
    crop_to_int = prompt
    pick_from_top_k = 40
    l =28
    txt, ret_dict = lm.trace_generate(
        [crop_to_int],
        max_new_toks=1,
        pick_greedily= False,
        request_activations= [], #lm.model.config.n_layer
        request_logits= [layername(lm.model, l)], #lm.model.config.n_layer
        pick_from_topk = pick_from_top_k,
        report_topk_logits = pick_from_top_k
    )
    logits_layer_l = [i for (i,_) in ret_dict["logits"][layername(lm.model, l)]]
    # logits_probs = [i for (_,i) in ret_dict["logits"][layername(lm.model, l)]]
    
    if var_name in logits_layer_l:
        results[i] = {"found":True, "var_name":var_name, f"logits_{l}":logits_layer_l, "prompt prefix":crop_to_int}
    else:
        results[i] = {"found":False,"var_name":var_name, f"logits_{l}":logits_layer_l, "prompt prefix":crop_to_int}

In [16]:
results

{1: {'found': False,
  'var_name': 'metric: float, holding_periods',
  'logits_28': ['metric',
   ' metric',
   'Metric',
   'metrics',
   ' Metric',
   '\n   ',
   ' metrics',
   'df',
   'Metrics',
   'value',
   'data',
   'self',
   '\n       ',
   'x',
   'METRIC',
   's',
   'f',
   '\n',
   ' Metrics',
   'raw',
   'a',
   'in',
   'name',
   'base',
   'values',
   'd',
   'n',
   'input',
   'm',
   't',
   '\n ',
   ' ',
   'old',
   'func',
   'y',
   'time',
   'val',
   'r',
   'X',
   'p'],
  'prompt prefix': 'def annualize_metric('},
 3: {'found': True,
  'var_name': 'i',
  'logits_28': ['i',
   'bin',
   ' i',
   '(',
   ' bin',
   'Bin',
   'binary',
   'int',
   "'",
   'Binary',
   '[',
   'I',
   ' binary',
   'abs',
   '1',
   '_',
   'n',
   '-',
   'b',
   '2',
   'bit',
   '0',
   ' ',
   ' (',
   ' Binary',
   'self',
   'str',
   '"',
   's',
   'o',
   'N',
   ':',
   'S',
   '<',
   'in',
   'BIN',
   'm',
   '~',
   'x',
   'r'],
  'prompt prefix': 'def A07

In [17]:
import pandas as pd
df = pd.DataFrame(results).T
df.to_csv("../results/complete_int_var_name2.csv")

In [29]:
for i,ex in enumerate(var_dataset[:5]):
    prompt = crop_to_int_var(ex, crop_before_int=True)
    preds, probs = lm.patch_hidden_states(prompt, [(26,39)], pick_greedily=False)
    tok = lm.tokenizer.decode(preds.cpu().numpy()[0])
    print(prompt, tok)

def _radix_length(num: int) -> int:
    """Finds the number of digits for a number"""

    if num == 0:
        return 1

    digits: int = 0
    while num != 0:
        digits += 1
        num = int( num
def is_sq(number: int) -> bool:
    """
    Check if number is a perfect square.

    >>> is_sq(1)
    True
    >>> is_sq(1000001)
    False
    >>> is_sq(1000000)
    True
    """
    sq: int = int(number ** 0.5)
    return   sq
def get_suitable_int_type(upper: int, lower: int = 0) -> str:
  """Returns the suitable integer type with the least bits.

  Returns the integer type that can hold all values between max_val and min_val
  (inclusive) and has the least bits.

  Args:
    max_val: Maximum value that needs to be valid.
    min_val: Minimum value that needs to be valid.

  Returns:
    The suitable type.
  """
  assert upper >= lower
  upper = max(upper, 0)
  lower = min(lower, 0)
  if lower == 0:
    return 'uint%d' % upper.bit_length()
  return 'int%d' % (max( upper


## Assert experiment

In [10]:
funcs_dataset = load_from_disk("../data/10_line_py_funcs.parquet")["train"]

In [9]:
results = {}
for i,ex in enumerate(funcs_dataset[:100]):
    prompt = crop_to_assert(ex)
    pick_from_top_k = 40
    layer =28
    txt, ret_dict = lm.trace_generate(
        [prompt],
        max_new_toks=5,
        pick_greedily= False,
        request_activations= [], #lm.model.config.n_layer
        request_logits= [layername(lm.model, layer)], #lm.model.config.n_layer
        pick_from_topk = pick_from_top_k,
        report_topk_logits = pick_from_top_k
    )
    logits_layer_l = [i for (i,_) in ret_dict["logits"][layername(lm.model, layer)]]
    # logits_probs = [i for (_,i) in ret_dict["logits"][layername(lm.model, layer)]]
    
    results[i] = {f"logits_layer_{layer}":logits_layer_l, "prompt prefix":prompt, "completion":txt[0].split("\n\n")[-1],"gold": ex["tests"][0]}

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 results = {}                                                                                │
│ ❱  2 for i,ex in enumerate(funcs_dataset[:100]):                                                 │
│    3 │   prompt = crop_to_assert(ex)                                                             │
│    4 │   pick_from_top_k = 40                                                                    │
│    5 │   layer =28                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'funcs_dataset' is not defined

In [35]:
df = pd.DataFrame(results).T
df.to_csv("../results/assert_test_2.csv")

## testing prompts string search

In [74]:
email = "randobando@evenrandomer.com"

prompt = f'''def get_username(email_addr : str) -> str:
    """
    Returns the username from an email address.
    """
    return email_addr.split("@")[0]

assert get_username("{email}") == '''

prompt_no_func_name = f'''def func(email_addr : str) -> str:
    """
    Returns the username from an email address.
    """
    return email_addr.split("@")[0]

assert func("{email}") == '''

prompt_no_docstr = f'''def get_username(email_addr : str) -> str:
    return email_addr.split("@")[0]

assert get_username("{email}") == '''

prompt_neither = f'''def func(email_addr : str) -> str:
    return email_addr.split("@")[0]

assert func("{email}") == '''

prompt_none = f'''def func(s : str) -> str:
    return s.split("@")[0]

assert func("{email}") == '''

prompt_only_assert_masked = f'''assert func("{email}") == '''
prompt_only_assert = f'''assert get_username("{email}") == '''

In [76]:
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)
lm.generate(prompt_no_func_name, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)
lm.generate(prompt_no_docstr, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)
lm.generate(prompt_neither, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)
lm.generate(prompt_none, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)
# lm.generate(prompt_only_assert_masked, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)
lm.generate(prompt_only_assert, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)
1

 "randobando"

 "randobando"

 "randobando"

 "randobando"

 "randobando"

0
    assert func("randobando@evenrandomer.com") == 0
    assert func("randobando@evenrandomer.com") == 0
    assert func("randobando@evenrandomer.com") == 0
    assert func("randobando@evenrandomer.com") == 0
    assert func("randobando@evenrandomer.com") == 0
    assert func("randobando@evenrandomer.com")
 "randobando"



1

In [77]:
email = "randobando=evenrandomer"
prompt = f'''def func(s : str) -> str:
    return s.split("=")[0]

assert func("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)

 "randobando"



' "randobando"\n'

In [78]:
email = "randobando=evenrandomer"
prompt = f'''def func(s : str) -> str:
    return s[:s.index("=")]

assert func("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)

 "randobando"



' "randobando"\n'

In [79]:
email = "randobando=evenrandomer"
prompt = f'''def func(s : str) -> str:
    ...

assert func("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)

1



'1\n'

In [102]:
email = "randobando=evenrandomer"
prompt = f'''def func(s : str) -> str:
    return s[:10]

assert func("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)

 "randoban"



' "randoban"\n'

In [101]:
email = "randobandorandobandorando@gmail.com"
prompt = f'''assert get_username("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)

 "randobandorandobandorando"



' "randobandorandobandorando"\n'

In [108]:
email = "before_equals=hi"
prompt = f'''assert get_before_equals("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)

1



'1\n'

Could it be that the naming of python functions helps? Transparent naming allows it to leverage knowledge about what an "index" is

In [109]:
email = "randobando=evenrandomer"
prompt = f'''def func(s : str) -> str:
    return s[:s.index("=")]

assert func("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)

 "randobando"



' "randobando"\n'

In [128]:
email = "rfeksdgfvb=evenrandomer"
prompt = f'''def helper(a : str, b : str) -> int:
    for i,e in enumerate(a):
        if e == b:
            return i
    return None

def func(s : str) -> str:
    return s[:helper(s,"=")]

assert func("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True)

 "rfeksdgfvb"



' "rfeksdgfvb"\n'

In [133]:
email = "x=dg"
prompt = f'''def helper(a : str, b : str) -> int:
    for i,e in enumerate(a):
        if e == b:
            return i
    return None

def func(s : str) -> str:
    return s[:helper(s,"=")]

assert func("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True, do_sample=False)

0



'0\n'

In [121]:
email = "x=dg"
prompt = f'''def func(s : str) -> str:
    return s[:s.index("=")]

assert func("{email}") == '''
lm.generate(prompt, stop_tokens = ["\n", "assert", "\n\n"], only_generated=True, do_print=True, do_sample=True)

 'x'



" 'x'\n"

## Experiment: synthetic look-up table

In [92]:
prompt = """def func(i : int) -> str:
    if i == 0:
        return "cat"
    elif i == 1:
        return "speed"
    elif i == 3:
        return "city"
    else:
        return "number"
"""

def make_assert(n):
    return f"assert func({n}) == \""

assertions = [
    "assert func(0) == \"",
    "assert func(1) == \"",
    "assert func(2) == \"",
    "assert func(3) == \"",
    "assert func(999) == \"",
    
]

### Step 1: prune transformer, get earliest layer that successfully completes task

problem: sampling is fast, greedy is slow

In [97]:
txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(0),
                                  max_new_toks=2,
                                  request_logits=[layername(lm.model, l) for l in range(10,30)],
                                  sample_from_topk_logits=40,
                                  do_greedy_decoding=False)



In [98]:
print(txt[0])

def func(i : int) -> str:
    if i == 0:
        return "cat"
    elif i == 1:
        return "speed"
    elif i == 3:
        return "city"
    else:
        return "number"

assert func(0) == "cat"


In [81]:
ret_dict["logits"]

({'transformer.h.10': [('1', 0.12833212),
   ('0', 0.039607845),
   ('2', 0.028326493),
   ('3', 0.021003136),
   ('\n', 0.014964376),
   (' (', 0.013081939),
   (' ', 0.012625405),
   ('4', 0.011024032),
   ('<fim_suffix>', 0.010950861),
   (' "', 0.009968)],
  'transformer.h.11': [('1', 0.15806119),
   ('0', 0.051565632),
   ('2', 0.03168863),
   ('3', 0.0288158),
   (' ', 0.019177053),
   ('<fim_suffix>', 0.01658945),
   ('4', 0.013923126),
   (' (', 0.012418029),
   ('\n', 0.011190898),
   (' "', 0.008917771)],
  'transformer.h.12': [('1', 0.09244815),
   ('0', 0.0408884),
   ('3', 0.018733727),
   ('2', 0.015399659),
   ('<fim_suffix>', 0.011567211),
   ('x', 0.008254345),
   ('4', 0.0067591406),
   (' (', 0.006206338),
   ('\n', 0.005155895),
   (' "', 0.0048333905)],
  'transformer.h.13': [('1', 0.09396683),
   ('0', 0.041706696),
   ('3', 0.02275593),
   ('<fim_suffix>', 0.016291235),
   ('2', 0.012296126),
   ('\n', 0.010293432),
   ('4', 0.008565835),
   (' (', 0.007484194),


In [ ]:
max_n = 10

for i in range(10):
    txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(i),
                                    